In [1]:
import requests
import re
import os
from bs4 import BeautifulSoup
from urllib.parse import urlsplit
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import underthesea #pip install underthesea


In [70]:
data=pd.read_excel('DATA_SQL.xlsx')
data

c:\Users\ntthu\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,ID,link,Tên sách,Tác giả,Thể loại,Giá bán,Giá gốc,Giảm giá,Số lượng bán,Mô tả sản phẩm,Điểm đánh giá,Số nhận xét
0,1000000001,https://tiki.vn/bikes-of-burden-hardcover-p556...,Bikes Of Burden - Hardcover,Hans Kemp,Architecture,575700,606000,-5%,42,Bikes Of Burden In Vietnam the motorbike is th...,5.0,2
1,1000000003,https://tiki.vn/southern-vietnamese-modernist-...,Southern Vietnamese Modernist Architecture (Tá...,Mel Schenck,Architecture,380000,380000,0%,12,Sách bao gồm những công trình hiện đại tiêu bi...,5.0,1
2,1000000005,https://tiki.vn/cabins-p138490437.html?itm_cam...,Cabins,Philip Jodidio,Architecture,650000,650000,0%,19,"Trong thập kỷ qua, khi sự tồn tại vật chất và ...",5.0,5
3,1000000006,https://tiki.vn/yes-is-more-p538480.html?itm_c...,Yes Is More,Bjarke Ingels,Architecture,760000,760000,0%,11,Yes Is More Action-packed architecture! A mani...,5.0,2
4,1000000007,https://tiki.vn/1000-architectural-details-p19...,1000 Architectural Details,Alex Sanchez Vidiella,Architecture,910000,910000,0%,0,1000 Chi tiết Kiến trúc là một danh mục tham k...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
41217,1000068147,https://tiki.vn/them-tuoi-them-duyen-cam-nang-...,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,Caroline De Maigret,Sách nghệ thuật sống đẹp,136000,160000,-15%,0,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,0.0,0
41218,1000068148,https://tiki.vn/combo-2-cuon-song-thanh-than-n...,Combo 2 cuốn: Sống thanh thản như người Thụy Đ...,Margareta Magnusson,Sách nghệ thuật sống đẹp,158400,176000,-10%,0,Sống thanh thản như người Thụy Điển Cuộc sống ...,0.0,0
41219,1000068151,https://tiki.vn/zero-waste-home-nha-khong-rac-...,Zero Waste Home: Nhà Không Rác (Tái Bản),Bea Johnson,Sách nghệ thuật sống đẹp,136000,155000,-12%,0,"“Môi trường, kinh tế và sức khỏe của chúng ta ...",0.0,0
41220,1000068152,https://tiki.vn/gia-dinh-tinh-thuc-p177264169....,GIA ĐÌNH TỈNH THỨC,Shefali Tsabary,Sách nghệ thuật sống đẹp,188100,209000,-10%,0,Gia Đình Tỉnh Thức là tài liệu tham khảo cực k...,0.0,0


In [71]:
def phathien(x):
    a=re.findall('[^0-9AÀẢÃÁẠĂẰẲẴẮẶÂẦẨẪẤẬBCDĐEÈẺẼÉẸÊỀỂỄẾỆFGHIÌỈĨÍỊJKLMNOÒỎÕÓỌÔỒỔỖỐỘƠỜỞỠỚỢPQRSTUÙỦŨÚỤƯỪỬỮỨỰVWXYỲỶỸÝỴZaàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnoòỏõóọôồổỗốộơờởỡớợpqrstuùủũúụưừửữứựvwxyỳỷỹýỵzAÀẢÃÁẠĂẰẲẴẮẶÂẦẨẪẤẬBCDĐEÈẺẼÉẸÊỀỂỄẾỆFGHIÌỈĨÍỊJKLMNOÒỎÕÓỌÔỒỔỖỐỘƠỜỞỠỚỢPQRSTUÙỦŨÚỤƯỪỬỮỨỰVWXYỲỶỸÝỴZaàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnoòỏõóọôồổỗốộơờởỡớợpqrstuùủũúụưừửữứựvwxyỳỷỹýỵzAÀẢÃÁẠĂẰẲẴẮẶÂẦẨẪẤẬBCDĐEÈẺẼÉẸÊỀỂỄẾỆFGHIÌỈĨÍỊJKLMNOÒỎÕÓỌÔỒỔỖỐỘƠỜỞỠỚỢPQRSTUÙỦŨÚỤƯỪỬỮỨỰVWXYỲỶỸÝỴZaàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnoòỏõóọôồổỗốộơờởỡớợpqrstuùủũúụưừửữứựvwxyỳỷỹýỵz\s\W_]',x)
    return a

In [72]:
for i in range(len(data)):
    if phathien(data['Tên sách'].iloc[i]) != []:
        data['Tên sách'].iloc[i]=np.nan
    if phathien(data['Tác giả'].iloc[i]) != []:
        data['Tác giả'].iloc[i]=np.nan
    if phathien(data['Mô tả sản phẩm'].iloc[i]) != []:
        data['Mô tả sản phẩm'].iloc[i]=np.nan
data.info()

C:\Users\ntthu\AppData\Local\Temp\ipykernel_1972\1909226710.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Mô tả sản phẩm'].iloc[i]=np.nan
C:\Users\ntthu\AppData\Local\Temp\ipykernel_1972\1909226710.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tác giả'].iloc[i]=np.nan
C:\Users\ntthu\AppData\Local\Temp\ipykernel_1972\1909226710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tên sách'].iloc[i]=np.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41222 entries, 0 to 41221
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 41222 non-null  int64  
 1   link               41222 non-null  object 
 2   Tên sách          40823 non-null  object 
 3   Tác giả          40749 non-null  object 
 4   Thể loại         41222 non-null  object 
 5   Giá bán          41222 non-null  int64  
 6   Giá gốc          41222 non-null  int64  
 7   Giảm giá         41222 non-null  object 
 8   Số lượng bán    41222 non-null  object 
 9   Mô tả sản phẩm  40227 non-null  object 
 10  Điểm đánh giá   41222 non-null  float64
 11  Số nhận xét     41222 non-null  int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 3.8+ MB


In [73]:
data.dropna(inplace=True)
data.to_excel('DataClean.xlsx', index=False)
data

,ID,link,Tên sách,Tác giả,Thể loại,Giá bán,Giá gốc,Giảm giá,Số lượng bán,Mô tả sản phẩm,Điểm đánh giá,Số nhận xét
0,1000000001,https://tiki.vn/bikes-of-burden-hardcover-p556...,Bikes Of Burden - Hardcover,Hans Kemp,Architecture,575700,606000,-5%,42,Bikes Of Burden In Vietnam the motorbike is th...,5.0,2
1,1000000003,https://tiki.vn/southern-vietnamese-modernist-...,Southern Vietnamese Modernist Architecture (Tá...,Mel Schenck,Architecture,380000,380000,0%,12,Sách bao gồm những công trình hiện đại tiêu bi...,5.0,1
2,1000000005,https://tiki.vn/cabins-p138490437.html?itm_cam...,Cabins,Philip Jodidio,Architecture,650000,650000,0%,19,"Trong thập kỷ qua, khi sự tồn tại vật chất và ...",5.0,5
3,1000000006,https://tiki.vn/yes-is-more-p538480.html?itm_c...,Yes Is More,Bjarke Ingels,Architecture,760000,760000,0%,11,Yes Is More Action-packed architecture! A mani...,5.0,2
4,1000000007,https://tiki.vn/1000-architectural-details-p19...,1000 Architectural Details,Alex Sanchez Vidiella,Architecture,910000,910000,0%,0,1000 Chi tiết Kiến trúc là một danh mục tham k...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
41216,1000068145,https://tiki.vn/sach-thien-that-ra-khong-kho-p...,Sách - Thiền Thật Ra Không Khó,Trần Luân Tín,Sách nghệ thuật sống đẹp,73500,73500,0%,0,Nhà xuất bản: NXB Dân Trí Tác giả: Trần Luân T...,0.0,0
41217,1000068147,https://tiki.vn/them-tuoi-them-duyen-cam-nang-...,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,Caroline De Maigret,Sách nghệ thuật sống đẹp,136000,160000,-15%,0,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,0.0,0
41219,1000068151,https://tiki.vn/zero-waste-home-nha-khong-rac-...,Zero Waste Home: Nhà Không Rác (Tái Bản),Bea Johnson,Sách nghệ thuật sống đẹp,136000,155000,-12%,0,"“Môi trường, kinh tế và sức khỏe của chúng ta ...",0.0,0
41220,1000068152,https://tiki.vn/gia-dinh-tinh-thuc-p177264169....,GIA ĐÌNH TỈNH THỨC,Shefali Tsabary,Sách nghệ thuật sống đẹp,188100,209000,-10%,0,Gia Đình Tỉnh Thức là tài liệu tham khảo cực k...,0.0,0


In [77]:
data=pd.read_excel('DataClean.xlsx')
data

,ID,link,Tên sách,Tác giả,Thể loại,Giá bán,Giá gốc,Giảm giá,Số lượng bán,Mô tả sản phẩm,Điểm đánh giá,Số nhận xét
0,1000000001,https://tiki.vn/bikes-of-burden-hardcover-p556...,Bikes Of Burden - Hardcover,Hans Kemp,Architecture,575700,606000,-5%,42,Bikes Of Burden In Vietnam the motorbike is th...,5.0,2
1,1000000003,https://tiki.vn/southern-vietnamese-modernist-...,Southern Vietnamese Modernist Architecture (Tá...,Mel Schenck,Architecture,380000,380000,0%,12,Sách bao gồm những công trình hiện đại tiêu bi...,5.0,1
2,1000000005,https://tiki.vn/cabins-p138490437.html?itm_cam...,Cabins,Philip Jodidio,Architecture,650000,650000,0%,19,"Trong thập kỷ qua, khi sự tồn tại vật chất và ...",5.0,5
3,1000000006,https://tiki.vn/yes-is-more-p538480.html?itm_c...,Yes Is More,Bjarke Ingels,Architecture,760000,760000,0%,11,Yes Is More Action-packed architecture! A mani...,5.0,2
4,1000000007,https://tiki.vn/1000-architectural-details-p19...,1000 Architectural Details,Alex Sanchez Vidiella,Architecture,910000,910000,0%,0,1000 Chi tiết Kiến trúc là một danh mục tham k...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
40145,1000068145,https://tiki.vn/sach-thien-that-ra-khong-kho-p...,Sách - Thiền Thật Ra Không Khó,Trần Luân Tín,Sách nghệ thuật sống đẹp,73500,73500,0%,0,Nhà xuất bản: NXB Dân Trí Tác giả: Trần Luân T...,0.0,0
40146,1000068147,https://tiki.vn/them-tuoi-them-duyen-cam-nang-...,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,Caroline De Maigret,Sách nghệ thuật sống đẹp,136000,160000,-15%,0,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,0.0,0
40147,1000068151,https://tiki.vn/zero-waste-home-nha-khong-rac-...,Zero Waste Home: Nhà Không Rác (Tái Bản),Bea Johnson,Sách nghệ thuật sống đẹp,136000,155000,-12%,0,"“Môi trường, kinh tế và sức khỏe của chúng ta ...",0.0,0
40148,1000068152,https://tiki.vn/gia-dinh-tinh-thuc-p177264169....,GIA ĐÌNH TỈNH THỨC,Shefali Tsabary,Sách nghệ thuật sống đẹp,188100,209000,-10%,0,Gia Đình Tỉnh Thức là tài liệu tham khảo cực k...,0.0,0


In [78]:
def read_file(text):	   
    return text.encode('utf-8')

# Tiền xử lý văn bản
import re
def Text_Preprocessing(doc):
    text_pre = doc.lower()
    
    text_pre = underthesea.word_tokenize(text_pre, format="text")
    text_pre = underthesea.text_normalize(text_pre)
    
    text_pre = text_pre.replace('XXXX', '')        
    text_pre = text_pre.replace(u'\ufffd', '')   # Replaces the ASCII symbol with ''
    text_pre = re.sub(r'[^\w\s]','',text_pre)
    text_pre = text_pre.rstrip('\n')              # Removes line breaks
    text_pre = text_pre.casefold()                # Makes all letters lowercase
    
    text_pre = re.sub('\W_',' ', text_pre)        # removes specials characters and leaves only words
    text_pre = re.sub("\S*\d\S*"," ", text_pre)   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
    text_pre = re.sub("\S*@\S*\s?"," ", text_pre) # removes emails and mentions (words with @)
    text_pre = re.sub(r'http\S+', '', text_pre)   # removes URLs with http
    text_pre = re.sub(r'www\S+', '', text_pre)    # removes URLs with www
        
        
    ##Remove stop words
    Path="D:/DAIHOC/KY6/QTCSDL"
    filename=os.path.join(Path, "vietnamese-stopwords.txt")
    f = open(filename, "r", encoding="utf-8")
    
    #Get Stop words Dictionaries
    List_StopWords=f.read().split("\n")
    text_pre=" ".join(text for text in text_pre.split() if text not in List_StopWords)
    
    return text_pre

In [66]:
doc_1=read_file(data['Mô tả sản phẩm'][0])
Text_Preprocessing(doc_1)

'bikes of burden in vietnam the motorbike is the main mode of transport not only for people but for every imaginable and unimaginable product and produce without the motorbike the economy would grind to a halt bikes of burden shows in stunning full color photographs how the motorbikes the drivers and their loads ride around the cities and countryside in acts that defy your wildest imagination a photographic record of life in modernday vietnam colorful and quirky it captures everyday life at street level the life of a city and its people giá sản_phẩm tiki bao_gồm thuế luật hiện_hành cạnh tùy sản_phẩm hình_thức địa_chỉ giao hàng có_thể phát_sinh chi_phí phí vận_chuyển phụ_phí hàng cồng_kềnh thuế nhập_khẩu đối_với đơn hàng giao nước_ngoài giá_trị triệu đồng'

In [80]:
data['keywordTS']=None
data['keywordTG']=None
data['keywordSP']=None
for i in range(len(data)):
    data['keywordTS'][i]=Text_Preprocessing(read_file(data['Tên sách'][i]))
    data['keywordTG'][i]=Text_Preprocessing(read_file(data['Tác giả'][i]))
    data['keywordSP'][i]=Text_Preprocessing(read_file(data['Mô tả sản phẩm'][i]))
    
data

C:\Users\ntthu\AppData\Local\Temp\ipykernel_1972\317039969.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywordTS'][i]=Text_Preprocessing(read_file(data['Tên sách'][i]))
C:\Users\ntthu\AppData\Local\Temp\ipykernel_1972\317039969.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywordTG'][i]=Text_Preprocessing(read_file(data['Tác giả'][i]))
C:\Users\ntthu\AppData\Local\Temp\ipykernel_1972\317039969.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

,ID,link,Tên sách,Tác giả,Thể loại,Giá bán,Giá gốc,Giảm giá,Số lượng bán,Mô tả sản phẩm,Điểm đánh giá,Số nhận xét,keywordTS,keywordTG,keywordSP
0,1000000001,https://tiki.vn/bikes-of-burden-hardcover-p556...,Bikes Of Burden - Hardcover,Hans Kemp,Architecture,575700,606000,-5%,42,Bikes Of Burden In Vietnam the motorbike is th...,5.0,2,bikes of burden hardcover,hans kemp,bikes of burden in vietnam the motorbike is th...
1,1000000003,https://tiki.vn/southern-vietnamese-modernist-...,Southern Vietnamese Modernist Architecture (Tá...,Mel Schenck,Architecture,380000,380000,0%,12,Sách bao gồm những công trình hiện đại tiêu bi...,5.0,1,southern vietnamese modernist architecture tái...,mel schenck,sách bao_gồm công_trình hiện_đại tiêu_biểu miề...
2,1000000005,https://tiki.vn/cabins-p138490437.html?itm_cam...,Cabins,Philip Jodidio,Architecture,650000,650000,0%,19,"Trong thập kỷ qua, khi sự tồn tại vật chất và ...",5.0,5,cabins,philip jodidio,thập_kỷ tồn_tại vật_chất dấu_ấn môi_trường chú...
3,1000000006,https://tiki.vn/yes-is-more-p538480.html?itm_c...,Yes Is More,Bjarke Ingels,Architecture,760000,760000,0%,11,Yes Is More Action-packed architecture! A mani...,5.0,2,yes is more,bjarke ingels,yes is more actionpacked architecture a manife...
4,1000000007,https://tiki.vn/1000-architectural-details-p19...,1000 Architectural Details,Alex Sanchez Vidiella,Architecture,910000,910000,0%,0,1000 Chi tiết Kiến trúc là một danh mục tham k...,0.0,0,architectural details,alex sanchez vidiella,chi_tiết kiến_trúc danh_mục tham_khảo đặc_điểm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40145,1000068145,https://tiki.vn/sach-thien-that-ra-khong-kho-p...,Sách - Thiền Thật Ra Không Khó,Trần Luân Tín,Sách nghệ thuật sống đẹp,73500,73500,0%,0,Nhà xuất bản: NXB Dân Trí Tác giả: Trần Luân T...,0.0,0,sách thiền thật_ra,trần luân tín,xuất_bản nxb dân_trí tác_giả trần luân tín xuấ...
40146,1000068147,https://tiki.vn/them-tuoi-them-duyen-cam-nang-...,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,Caroline De Maigret,Sách nghệ thuật sống đẹp,136000,160000,-15%,0,Thêm Tuổi Thêm Duyên - Cẩm Nang Sống Chất Của ...,0.0,0,duyên cẩm_nang sống chất phụ_nữ pháp,caroline de maigret,duyên cẩm_nang sống chất phụ_nữ pháp suy_nghĩ ...
40147,1000068151,https://tiki.vn/zero-waste-home-nha-khong-rac-...,Zero Waste Home: Nhà Không Rác (Tái Bản),Bea Johnson,Sách nghệ thuật sống đẹp,136000,155000,-12%,0,"“Môi trường, kinh tế và sức khỏe của chúng ta ...",0.0,0,zero waste home rác tái_bản,bea johnson,môi_trường kinh_tế sức_khỏe chúng_ta khủng_hoả...
40148,1000068152,https://tiki.vn/gia-dinh-tinh-thuc-p177264169....,GIA ĐÌNH TỈNH THỨC,Shefali Tsabary,Sách nghệ thuật sống đẹp,188100,209000,-10%,0,Gia Đình Tỉnh Thức là tài liệu tham khảo cực k...,0.0,0,gia_đình tỉnh thức,shefali tsabary,gia_đình tỉnh thức tài_liệu tham_khảo cực_kì_h...


In [81]:
data.to_excel('KEYWORD.xlsx', index=False)